This model will try out attention with the original keras seq2seq model without embedding. try transformer maybe graph2seq, try reversing the training sentence

In [1]:
% cd .. 
% cd root
import numpy as np
import pandas as pd
df = pd.read_csv("AL_clean_2.csv")
df.head()
df.fillna('', inplace=True)
print(len(df))
x = df["x1"].tolist()
y = df["y1"].tolist()

/
/root
56820


In [2]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.

# Vectorize the data.
input_texts = []
target_texts = []
input_words = set()
target_words = set()

for line in x:
    input_texts.append(['\t']+ str(line).split(" ")+ ['\n'])
    for word in str(line).split(" "):
        if word not in input_words:
            input_words.add(word)

for line in y:
  target_texts.append(['\t'] + str(line).split(" ") + ['\n'])
  for word in str(line).split(" "):
    if word not in target_words:
        target_words.add(word)
target_words.add("\t")
target_words.add("\n")
input_words.add("\t")
input_words.add("\n")

input_words = sorted(list(input_words))
target_words = sorted(list(target_words))
num_encoder_tokens = len(input_words)
num_decoder_tokens = len(target_words)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Using TensorFlow backend.


Number of samples: 56820
Number of unique input tokens: 7747
Number of unique output tokens: 7652
Max sequence length for inputs: 27
Max sequence length for outputs: 27


In [0]:
input_token_index = dict(
    [(word, i) for i, word in enumerate(input_words)])
target_token_index = dict(
    [(word, i) for i, word in enumerate(target_words)])

In [4]:
import tensorflow as tf
input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length),
    dtype='float32')
output_data = np.zeros(
    (len(input_texts), max_decoder_seq_length),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, word in enumerate(input_text):
        input_data[i, t] = input_token_index[word]
    for t, word in enumerate(target_text):
        output_data[i, t] = target_token_index[word]

dataset = tf.data.Dataset.from_tensor_slices((input_data, output_data)).shuffle(len(input_data)).batch(64, drop_remainder=True)
print(input_data.shape)

(56820, 27)


In [5]:
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([64, 27]), TensorShape([64, 27]))

In [0]:
#https://www.tensorflow.org/tutorials/text/nmt_with_attention

class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.lstm = tf.keras.layers.GRU(self.enc_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')

  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.lstm(x, initial_state = hidden)
    return output, state

  def initialize_hidden_state(self):
    return [tf.zeros((self.batch_sz, self.enc_units))]


class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, query, values):
    # query hidden state shape == (batch_size, hidden size)
    # query_with_time_axis shape == (batch_size, 1, hidden size)
    # values shape == (batch_size, max_len, hidden size)
    # we are doing this to broadcast addition along the time axis to calculate the score
    query_with_time_axis = tf.expand_dims(query, 1)

    # score shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
    score = self.V(tf.nn.tanh(
        self.W1(query_with_time_axis) + self.W2(values)))

    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.lstm = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)

    # used for attention
    self.attention = BahdanauAttention(self.dec_units)

  def call(self, x, hidden, enc_output):
    # enc_output shape == (batch_size, max_length, hidden_size)
    context_vector, attention_weights = self.attention(hidden, enc_output)

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # passing the concatenated vector to the GRU
    output, state = self.lstm(x)
    # output shape == (batch_size * 1, hidden_size)
    output = tf.reshape(output, (-1, output.shape[2]))

    # output shape == (batch_size, vocab)
    x = self.fc(output)

    return x, state, attention_weights

In [0]:
encoder = Encoder(num_encoder_tokens, 100, 1024, 64)
decoder = Decoder(num_decoder_tokens, 100, 1024, 64)
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

In [0]:
def train_step(inp, targ, enc_hidden):
  loss = 0
  with tf.GradientTape() as tape:
    enc_output, enc_hidden = encoder(inp, enc_hidden)
    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([target_token_index['\t']] * BATCH_SIZE, 1)
    # Teacher forcing - feeding the target as the next input
    for t in range(1, targ.shape[1]):
      # passing enc_output to the decoder
      predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
      loss += loss_function(targ[:, t], predictions)
      # using teacher forcing
      dec_input = tf.expand_dims(targ[:, t], 1)
  batch_loss = (loss / int(targ.shape[1]))
  variables = encoder.trainable_variables + decoder.trainable_variables
  gradients = tape.gradient(loss, variables)
  optimizer.apply_gradients(zip(gradients, variables))
  return batch_loss

In [9]:
EPOCHS = 10
BATCH_SIZE = 64
embedding_dim = 256
units = 1024


import time
steps_per_epoch = len(input_data)//64
for epoch in range(EPOCHS):
  start = time.time()
  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0
  for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
    batch_loss = train_step(inp, targ, enc_hidden)
    total_loss += batch_loss
    if batch % 100 == 0:
      print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                   batch,
                                                   batch_loss.numpy()))
  print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
  print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 3.0119
Epoch 1 Batch 100 Loss 2.2889
Epoch 1 Batch 200 Loss 1.7755
Epoch 1 Batch 300 Loss 1.6993
Epoch 1 Batch 400 Loss 1.5176
Epoch 1 Batch 500 Loss 1.6933
Epoch 1 Batch 600 Loss 1.9126
Epoch 1 Batch 700 Loss 1.7851
Epoch 1 Batch 800 Loss 1.6418
Epoch 1 Loss 1.8130
Time taken for 1 epoch 408.3906192779541 sec

Epoch 2 Batch 0 Loss 1.7055
Epoch 2 Batch 100 Loss 1.6482
Epoch 2 Batch 200 Loss 1.4949
Epoch 2 Batch 300 Loss 1.7221
Epoch 2 Batch 400 Loss 1.5354
Epoch 2 Batch 500 Loss 1.6502
Epoch 2 Batch 600 Loss 1.5362
Epoch 2 Batch 700 Loss 1.5880
Epoch 2 Batch 800 Loss 1.7002
Epoch 2 Loss 1.6345
Time taken for 1 epoch 401.6073362827301 sec

Epoch 3 Batch 0 Loss 1.6256
Epoch 3 Batch 100 Loss 1.5030
Epoch 3 Batch 200 Loss 1.6165
Epoch 3 Batch 300 Loss 1.4334
Epoch 3 Batch 400 Loss 1.3699
Epoch 3 Batch 500 Loss 1.4212
Epoch 3 Batch 600 Loss 1.3734
Epoch 3 Batch 700 Loss 1.3903
Epoch 3 Batch 800 Loss 1.6847
Epoch 3 Loss 1.5559
Time taken for 1 epoch 400.10047006607056 se

In [0]:
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [0]:
def evaluate(sentence):
  attention_plot = np.zeros((max_decoder_seq_length, max_encoder_seq_length))

  inputs = [input_token_index[i] for i in sentence.split(' ')]
  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                         maxlen=max_encoder_seq_length,
                                                         padding='post')
  inputs = tf.convert_to_tensor(inputs)
  result = ''
  hidden = [tf.zeros((1, units))]
  enc_out, enc_hidden = encoder(inputs, hidden)
  dec_hidden = enc_hidden
  dec_input = tf.expand_dims([target_token_index['\t']], 0)
  for t in range(max_decoder_seq_length):
    predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                         dec_hidden,
                                                         enc_out)

    # storing the attention weights to plot later on
    attention_weights = tf.reshape(attention_weights, (-1, ))
    attention_plot[t] = attention_weights.numpy()

    predicted_id = tf.argmax(predictions[0]).numpy()

    result += reverse_target_char_index[predicted_id] + ' '

    if reverse_target_char_index[predicted_id] == '\n':
      return result, sentence, attention_plot

    # the predicted ID is fed back into the model
    dec_input = tf.expand_dims([predicted_id], 0)

  return result, sentence, attention_plot

In [0]:
def translate(sentence):
  result, sentence, attention_plot = evaluate(sentence)

  print('Input: %s' % (sentence))
  print('Predicted translation: {}'.format(result))


In [14]:
testing = pd.read_csv("testing.csv", header=None)
tests = testing[0].tolist()
for test in tests:
  translate(test)

Input: i am an amazon warrior it means i shop on amazon a lot
Predicted translation: what is your job 
 
Input: how are you doing today
Predicted translation: i am feeling down 
 
Input: i do not like you because of your face
Predicted translation: what is your job 
 
Input: what do you think about trump
Predicted translation: i have a lot of a lot of a lot of a lot of a lot of a lot of a lot of a lot of a 
Input: i suppose you are smart can you make me a sandwich
Predicted translation: i am 
 
Input: can you do math on your own
Predicted translation: i do not to do a lot of a different 
 
Input: so i got a girlfriend today
Predicted translation: what is that is the pain 
 
Input: what kind of pizza do you like
Predicted translation: i like the hobby and i like the hobby and i like the hobby and i like the hobby and i like the hobby and i like 
Input: i am working on a website project 
Predicted translation: what is your job 
 
Input: do you like chem homework
Predicted translation: i 